## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-29-02-41-40 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/28/us/min...
1,2026-01-29-02-40-55 +0000,nyt,The Journey of a Group of Cuban Deportees Stuc...,https://www.nytimes.com/2026/01/28/us/politics...
2,2026-01-29-02-35-07 +0000,bbc,Tesla sees first annual revenue drop as it shi...,https://www.bbc.com/news/articles/c620177qdg5o...
3,2026-01-29-02-32-00 +0000,wsj,Judge Temporarily Bars Trump Administration Fr...,https://www.wsj.com/politics/policy/judge-temp...
4,2026-01-29-02-27-00 +0000,wsj,Samsung Profit Beats as Chip Earnings Soar on ...,https://www.wsj.com/business/earnings/samsungs...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2416/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,48
123,minneapolis,20
55,ice,19
227,iran,17
279,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2026-01-29-02-07-32 +0000,nyt,The Trump administration moves to sell a landm...,https://www.nytimes.com/live/2026/01/28/us/tru...,124
119,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...,118
202,2026-01-28-18-58-42 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...,117
98,2026-01-28-23-10-05 +0000,bbc,Watch: How Alex Pretti shooting led Trump to s...,https://www.bbc.com/news/videos/cj6wgelw62do?a...,113
309,2026-01-28-12-30-42 +0000,nypost,"Trump calls GOP Sens. Murkowski, Tillis ‘loser...",https://nypost.com/2026/01/28/us-news/gop-sens...,112


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,124,2026-01-29-02-07-32 +0000,nyt,The Trump administration moves to sell a landm...,https://www.nytimes.com/live/2026/01/28/us/tru...
184,92,2026-01-28-19-46-10 +0000,cbc,2 federal agents involved in shooting of Alex ...,https://www.cbc.ca/news/world/agents-pretti-sh...
119,70,2026-01-28-22-21-00 +0000,wsj,President Trump rose to power following a simp...,https://www.wsj.com/politics/policy/trump-and-...
202,69,2026-01-28-18-58-42 +0000,bbc,Trump warns Iran 'time is running out' for nuc...,https://www.bbc.com/news/articles/cly5pd459gko...
36,57,2026-01-29-01-00-00 +0000,wsj,Minneapolis Police and Residents Forge New Com...,https://www.wsj.com/us-news/minneapolis-police...
229,55,2026-01-28-17-30-00 +0000,wsj,The Bank of Canada kept its policy rate unchan...,https://www.wsj.com/articles/bank-of-canada-st...
378,51,2026-01-28-06-26-29 +0000,nypost,South Korea’s former first lady Kim Keon Hee s...,https://nypost.com/2026/01/28/world-news/south...
199,46,2026-01-28-19-00-19 +0000,nypost,Fed holds interest rates steady despite Trump ...,https://nypost.com/2026/01/28/business/fed-hol...
371,46,2026-01-28-08-04-36 +0000,bbc,Zelensky condemns Russian drone strike that ki...,https://www.bbc.com/news/articles/crkrpl4ngdzo...
375,40,2026-01-28-07-11-07 +0000,nypost,"Private plane crashes in India, killing Deputy...",https://nypost.com/2026/01/28/world-news/india...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
